# ODP Data Analysis

This notebook analyze the data from https://www.europeandataportal.eu/. The dump of the portal was done in February 2018 and contains 804,982 datasets. To bootstrap the repository you will need to download the dataset from public HOBBIT FTP server (13G unpacked):
```
make get-data
```
After that, we will filter out German datasets.

In [1]:
# necessary imports
import pandas as pd
import json
import numpy as np
import copy

pd.options.display.max_rows = 4000
pd.options.display.max_columns = 4000

In [9]:
data_file = "data/europeandataportal.jsonl"
datasets_to_analyze = 1000

# Find out by which string we can filter out German datasets
# flatten out language field

def flatten_lang(languages):
    flat_languages = []
    for lang in languages:
        first_key = list(lang.keys())[0]
        flat_languages.append(lang.get(first_key))
    return flat_languages

def flatten_languages(dataset):
    new_dataset = copy.deepcopy(dataset)
    if "language" in dataset.keys():
        languages = flatten_lang(dataset["language"])
        for lang in languages:
            new_dataset[lang] = True
    return new_dataset

json_datasets = []
i = 0
for line in open(data_file, 'r'):
    dataset = json.loads(line)
    json_datasets.append(flatten_languages(dataset))
    i += 1
    if i == datasets_to_analyze:
        break

odp_data = pd.DataFrame(json_datasets)

np.random.seed(0)
odp_data.sample(2)

# How many values are missing ?
missing_values_count = odp_data.isnull().sum()

print(missing_values_count)

[{'label': 'en'}]
access_rights                                                     717
accrual_periodicity                                               537
author                                                           1000
author_email                                                     1000
conforms_to                                                       779
contact_point                                                     410
creator_user_id                                                     0
dct_type                                                          405
en                                                                995
extras                                                              0
groups                                                              0
has_version                                                       717
http://publications.europa.eu/resource/authority/language/DAN     997
http://publications.europa.eu/resource/authority/language/DUT     962
ht

# Filtering out German datasets
As we can see from the snippet above we can filter out the German datasets by special URL used by German government bodies. This type of filtering results in 184,724 datasets, while the data portal lists 205,851. We further filter remaining datasets by "www.govdata.de" string and get additional 35,044 datasets. The whole routine can be run with:
```
make filter-german-datasets
```

# Analysis of German datasets
We can sample both portions of the German datasets by taking 1000 from each and take a look at the missing values.

In [19]:
german_data_path_1 = "data/europeandataportal-ger-1.jsonl"
german_data_path_2 = "data/europeandataportal-ger-2.jsonl"

def load_json(data_path, lines_to_load):
    json_datasets = []
    i = 0
    for line in open(data_path, 'r'):
        dataset = json.loads(line)
        json_datasets.append(dataset)
        i += 1
        if i == lines_to_load:
            break
    return json_datasets

datasets_to_analyze = 1000
# german_data_1 is filtered by language URL
german_data_1 = pd.DataFrame(load_json(german_data_path_1, datasets_to_analyze))
# german_data_2 is filtered by www.govdata.de URL
german_data_2 = pd.DataFrame(load_json(german_data_path_2, datasets_to_analyze))

In [28]:
# Print out available columns
print(pd.DataFrame(german_data_1.columns.values))

                           0
0              access_rights
1        accrual_periodicity
2                     author
3               author_email
4                conforms_to
5              contact_point
6            creator_user_id
7                   dct_type
8                     extras
9                     groups
10               has_version
11                        id
12                identifier
13             is_version_of
14                    isopen
15                    issued
16                  language
17                license_id
18             license_title
19                maintainer
20          maintainer_email
21          metadata_created
22         metadata_modified
23                  modified
24                      name
25                     notes
26             num_resources
27                  num_tags
28              organization
29          other_identifier
30                 owner_org
31                      page
32                   private
33            

In [29]:
# Print out available columns
print(pd.DataFrame(german_data_2.columns.values))

                           0
0                     author
1               author_email
2              contact_point
3            creator_user_id
4                     extras
5                     groups
6                         id
7                 identifier
8                     isopen
9                 license_id
10             license_title
11               license_url
12                maintainer
13          maintainer_email
14          metadata_created
15         metadata_modified
16                  modified
17                      name
18                     notes
19             num_resources
20                  num_tags
21              organization
22           original_source
23                 owner_org
24                   private
25                 publisher
26   relationships_as_object
27  relationships_as_subject
28                 resources
29               revision_id
30                     state
31                      tags
32                  temporal
33            

In [35]:
# Print the columns in german_data_1 which are not in german_data_2
print(set(german_data_1.columns.values) - set(german_data_2.columns.values))

{'has_version', 'dct_type', 'other_identifier', 'provenance', 'accrual_periodicity', 'is_version_of', 'page', 'relation', 'language', 'issued', 'access_rights', 'conforms_to'}


In [36]:
# Print the columns in german_data_2 which are not in german_data_1
print(set(german_data_2.columns.values) - set(german_data_1.columns.values))

{'license_url', 'original_source'}


In [42]:
# Print common columns
common_columns = set(german_data_2.columns.values).intersection(set(german_data_1.columns.values))
common_columns = list(intersection)
common_columns.sort()
print(pd.DataFrame(common_columns))

                           0
0                     author
1               author_email
2              contact_point
3            creator_user_id
4                     extras
5                     groups
6                         id
7                 identifier
8                     isopen
9                 license_id
10             license_title
11                maintainer
12          maintainer_email
13          metadata_created
14         metadata_modified
15                  modified
16                      name
17                     notes
18             num_resources
19                  num_tags
20              organization
21                 owner_org
22                   private
23                 publisher
24   relationships_as_object
25  relationships_as_subject
26                 resources
27               revision_id
28                     state
29                      tags
30                  temporal
31                     title
32                translated
33            

In [46]:
# How many values are missing ?
missing_values_count_1 = german_data_1.isnull().sum()

print(missing_values_count_1)

access_rights                597
accrual_periodicity          254
author                      1000
author_email                1000
conforms_to                  922
contact_point                254
creator_user_id                0
dct_type                     403
extras                         0
groups                         0
has_version                  597
id                             0
identifier                     0
is_version_of                597
isopen                         0
issued                       272
language                       0
license_id                  1000
license_title               1000
maintainer                  1000
maintainer_email            1000
metadata_created               0
metadata_modified              0
modified                     155
name                           0
notes                          0
num_resources                  0
num_tags                       0
organization                   0
other_identifier             597
owner_org 

In [45]:
# How many values are missing ?
missing_values_count_2 = german_data_2.isnull().sum()

print(missing_values_count_2)

author                      1000
author_email                1000
contact_point                 21
creator_user_id                0
extras                         0
groups                         0
id                             0
identifier                     0
isopen                         0
license_id                     0
license_title                  0
license_url                  596
maintainer                  1000
maintainer_email            1000
metadata_created               0
metadata_modified              0
modified                     466
name                           0
notes                          0
num_resources                  0
num_tags                       0
organization                   0
original_source                0
owner_org                      0
private                        0
publisher                    682
relationships_as_object        0
relationships_as_subject       0
resources                      0
revision_id                    0
state     

In [49]:
total_cells_1 = np.product(german_data_1.shape)
total_missing_1 = missing_values_count_1.sum()

# percent of data that is missing
print("Percentage of missing values in cells for ger-1")
print((total_missing_1/total_cells_1) * 100)

total_cells_2 = np.product(german_data_2.shape)
total_missing_2 = missing_values_count_2.sum()

# percent of data that is missing
print("Percentage of missing values in cells for ger-2")
print((total_missing_2/total_cells_2) * 100)

Percentage of missing values in cells for ger-1
29.121999999999996
Percentage of missing values in cells for ger-2
19.4475
